# Install GPU related stuff

In [1]:
!pip install cudf cuml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 MB 16.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.12.2 requires dask<=2021.11.2,>=2021.11.1, but you have dask 2022.2.0 which is incompatible.


# Import necessary libraries

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import cudf
import cuml
from cuml.model_selection import train_test_split
from cuml.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tumai-x-aimunich-x-kkcompany/train_target_events.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/sample.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/light_train_target_labels.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/light_train_source_labels.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/test_source_events.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/train_source_events.csv
/kaggle/input/tumai-x-aimunich-x-kkcompany/light_test_source_labels.csv


# Prepare Data

In [3]:
time_slot_data_X = cudf.read_csv('/kaggle/input/tumai-x-aimunich-x-kkcompany/light_train_source_labels.csv')
time_slot_data_y = cudf.read_csv('/kaggle/input/tumai-x-aimunich-x-kkcompany/light_train_target_labels.csv')

In [4]:
time_slot_cols_X = list(time_slot_data_X.columns)
time_slot_cols_X.remove('user_id')

time_slot_cols_y = list(time_slot_data_y.columns)
time_slot_cols_y.remove('user_id')


X = time_slot_data_X[time_slot_cols_X]
y = time_slot_data_y[time_slot_cols_y]

# split into training and testing sets
X_train, X_test, y_train_cudf, y_test_cudf = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = cudf.DataFrame(X_train)
X_test = cudf.DataFrame(X_test)

y_train = cudf.DataFrame(y_train_cudf)
y_test = cudf.DataFrame(y_test_cudf)

# Statistical Analysis

In [5]:
#slots_per_week = 28
#num_weeks = X_train.shape[1] / slots_per_week
#shape = X_train.shape

#weeks = []
#for i in range(int(num_weeks)):
#    weeks.append(X_train.iloc[1][(28*i):(28*i)+28])

#week = np.zeros(slots_per_week)
#for i in range(len(weeks)):
#    week += weeks[i].to_numpy()

#week = week / num_weeks

    
#week =  [1 if value >= 0.2 else 0 for value in week]


In [6]:
#slots_per_week = 28
#num_weeks = X_test.shape[1] / slots_per_week
#shape = X_test.shape

#prob = pd.DataFrame()
#for j in range(shape[0]):
 #   weeks = []
  #  for i in range(int(num_weeks)):
  #      weeks.append(X_train.iloc[1][(28*i):(28*i)+28])

#    week = np.zeros(slots_per_week)
#    for i in range(len(weeks)):
#        week += weeks[i].to_numpy()

#    week = week / num_weeks

    
#    week =  [1 if value >= 0.2 else 0 for value in week]
    
 #   prob.append(week)

In [7]:
#print(week)

In [8]:
def stat_analysis(X_test):
    slots_per_week = 28
    num_weeks = X_test.shape[1] / slots_per_week
    shape = X_test.shape

    prob = []
    for j in range(shape[0]):
        weeks = []
        for i in range(int(num_weeks)):
            weeks.append(X_train.iloc[1][(28*i):(28*i)+28])

        week = np.zeros(slots_per_week)
        for i in range(len(weeks)):
            week += weeks[i].to_numpy()

        week = week / num_weeks


        week =  [1 if value >= 0.2 else 0 for value in week]

        prob.append(week)
        return prob

# Classification NN

In [9]:
from sklearn.neural_network import MLPClassifier

start_time = time.time()

nn_clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(500, 200, 50), verbose = True, random_state=1, max_iter=100)
nn_clf.fit(X_train.to_numpy(), y_train.to_numpy())

# make predictions on the testing data
y_pred = nn_clf.predict(X_test.to_numpy())

# calculate accuracy of the predictions
accuracy = cuml.accuracy_score(y_test.values.ravel(), y_pred)

end_time = time.time()


print('Accuracy:', accuracy)
print("Execution time: ", end_time - start_time,"secs")

Iteration 1, loss = 10.33600095
Iteration 2, loss = 8.57249609
Iteration 3, loss = 8.18249200
Iteration 4, loss = 7.81113148
Iteration 5, loss = 7.37467709
Iteration 6, loss = 6.85659727
Iteration 7, loss = 6.27422425
Iteration 8, loss = 5.64526185
Iteration 9, loss = 5.03474793
Iteration 10, loss = 4.43218859
Iteration 11, loss = 3.88169341
Iteration 12, loss = 3.39139848
Iteration 13, loss = 2.96016024
Iteration 14, loss = 2.61283344
Iteration 15, loss = 2.30982454
Iteration 16, loss = 2.05168548
Iteration 17, loss = 1.82147938
Iteration 18, loss = 1.64014183
Iteration 19, loss = 1.47357721
Iteration 20, loss = 1.35639208
Iteration 21, loss = 1.22906561
Iteration 22, loss = 1.14733609
Iteration 23, loss = 1.07231874
Iteration 24, loss = 0.98533698
Iteration 25, loss = 0.92653826
Iteration 26, loss = 0.86281816
Iteration 27, loss = 0.80879697
Iteration 28, loss = 0.73938975
Iteration 29, loss = 0.71485386
Iteration 30, loss = 0.71425282
Iteration 31, loss = 0.71099173
Iteration 32, lo

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Accuracy: 0.7173342108726501
Execution time:  384.93069553375244 secs


In [10]:
accuracy = metrics.roc_auc_score(y_test.to_numpy(), y_pred)
print(accuracy)

0.5833566753257574


# Hyperparameter Tuning

In [11]:
#param_grid = {
#    "max_depth": [3, 4, 5, 7],
#    "learning_rate": [0.1, 0.01, 0.05],
#    "gamma": [0, 0.25, 1],
#    "reg_lambda": [0, 1, 10],
#    "scale_pos_weight": [1, 3, 5],
#    "subsample": [0.8],
#    "colsample_bytree": [0.5],
#}

In [12]:
#from sklearn.model_selection import GridSearchCV

#start_time = time.time()

# Init classifier
#xgb_cl = xgb.XGBClassifier(objective="binary:logistic", tree_method='gpu_hist', gpu_id=0)

# Init Grid Search
#grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")

# Fit
#grid_cv.fit(X_train.to_numpy(), y_train.to_numpy())

#end_time = time.time()

#print("Execution time: ", end_time - start_time,"secs")

In [13]:
#grid_cv.best_score_
#grid_cv.best_params_

# Train XG-Boost Classifier

In [14]:
#start_time = time.time()
# create XGBoost classifier
#clf = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0)

# fit the classifier to the training data
#clf.fit(X_train, y_train)

# make predictions on the testing data
#y_pred = clf.predict(X_test)

# calculate accuracy of the predictions
#accuracy = cuml.accuracy_score(y_test.values.ravel(), y_pred)

#end_time = time.time()

#print('Accuracy:', accuracy)
#print("Execution time: ", end_time - start_time,"secs")

In [15]:
#accuracy = metrics.roc_auc_score(y_test.to_numpy(), y_pred)
#print(accuracy)

# Create Submission

In [16]:
X_submission = cudf.read_csv('/kaggle/input/tumai-x-aimunich-x-kkcompany/light_test_source_labels.csv')
user_ids = X_submission['user_id']
X_sub = X_submission[time_slot_cols_X]
y_sub = nn_clf.predict(X_sub.to_numpy())
#y_sub = stat_analysis(X_sub)
submission = pd.DataFrame(y_sub, columns=time_slot_cols_y)
submission['user_id'] = user_ids.to_numpy()
submission = submission[['user_id']+time_slot_cols_y]

In [17]:
submission.to_csv('submission.csv', index=False)